In [140]:
import csv
import pandas as pd
import tweepy
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
nltk.download('stopwords')
stop = stopwords.words('english')
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anubu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
auth = tweepy.AppAuthHandler('1d65XdPytohEvblDsR7JtSPb6', 'NhdYuQSflrMw9aqbl8LhLeh6iKaOejQC5F1nlMKVF1ONTjFxc7')
hashtag = "#ETH"
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
#Downloading data from Twitter and moving it over to a CSV file
#csvFile = open('Teaching group.csv', 'a')
#csvWriter = csv.writer(csvFile)
#csvWriter.writerow(['Tweet date', 'Tweet'])
#new_search = hashtag + ' -filter:retweets' + ' -filter:replies'
#for tweet in tweepy.Cursor(api.search, q = new_search, count = 50, lang = 'en', since_id = 0).items(100):
#    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
#csvFile.close()

In [3]:
with open('Teaching group.csv', 'r', encoding = 'utf-8', newline = '\n') as csvfile:
    csvreader = csv.reader(csvfile)
    date = []
    tweetText = []
    for row in csvreader:
        date.append(row[0])
        tweetText.append(row[1])
pandasTweet = pd.read_csv('Teaching group.csv')

In [144]:
recentDate = date[81]
oldestDate = date[-1]


2021-06-07 15:43:47
2021-06-07 15:38:40


In [5]:
#Analysis
pandasTweet['word_count'] = pandasTweet['Tweet'].apply(lambda x: len(str(x).split(" ")))
pandasTweet[['Tweet','word_count']].head()

,Tweet,word_count
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,10
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,12
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",21
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",19
4,"b""By buying tokens of our 100's property locat...",20


In [6]:
pandasTweet['char_count'] = pandasTweet['Tweet'].str.len()
pandasTweet[['Tweet','char_count']].head()

,Tweet,char_count
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,155
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,145
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",154
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",157
4,"b""By buying tokens of our 100's property locat...",158


In [7]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

pandasTweet['avg_word'] = pandasTweet['Tweet'].apply(lambda x: avg_word(x))
pandasTweet[['Tweet','avg_word']].head()

,Tweet,avg_word
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,14.600000
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,11.166667
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",6.700000
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",7.315789
4,"b""By buying tokens of our 100's property locat...",6.950000


In [8]:
pandasTweet['stopwords'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
pandasTweet[['Tweet','stopwords']].head()

,Tweet,stopwords
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,0
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,0
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",7
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",1
4,"b""By buying tokens of our 100's property locat...",6


In [9]:
pandasTweet['hastags'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
pandasTweet[['Tweet','hastags']].head()

,Tweet,hastags
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,1
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,2
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",3
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",5
4,"b""By buying tokens of our 100's property locat...",2


In [10]:
pandasTweet['numerics'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
pandasTweet[['Tweet','numerics']].head()

,Tweet,numerics
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,0
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,0
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",0
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",0
4,"b""By buying tokens of our 100's property locat...",0


In [11]:
pandasTweet['upper'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
pandasTweet[['Tweet','upper']].head()

,Tweet,upper
0,b'New CoinMarketCap Listing!\n\nEscoinToken / ...,0
1,b'#Ethereum hourly net stats.\n3pm - 4pm UTC\n...,1
2,"b""#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...",2
3,"b""$ALBT doesn't stop!!! #DEX #AMM #DEFI \n\n$Q...",12
4,"b""By buying tokens of our 100's property locat...",1


In [34]:
#Text cleanup/normalization
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'\\n', ' ', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'\\\w+?', '', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'https:\S+', '', regex=True)
pandasTweet['Tweet']

0     b'New CoinMarketCap Listing!  EscoinToken / $E...
1     b'#Ethereum hourly net stats. 3pm - 4pm UTC bl...
2     b"#DOGGYSWAP #ETH #BTC #CRYPTO..This project i...
3     b"$ALBT doesn't stop!!! #DEX #AMM #DEFI   $QNT...
4     b"By buying tokens of our 100's property locat...
                            ...                        
95    b'Membalas  @layer1crypto Great project, I hop...
96    b'All About #ApeSwap BUIDL Program  f09fa4ab J...
97    b'#BTC #ETH #DOGE  are the future. We are goin...
98    b"While this sideways action is happening, it'...
99    b"How did you get into Crypto?  Here's how I g...
Name: Tweet, Length: 100, dtype: object

In [35]:
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('[^\w\s]','')
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: '' + x[1:])
pandasTweet['Tweet']

<ipython-input-35-2fe78207170a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('[^\w\s]','')


0     new coinmarketcap listing escointoken elg pric...
1     ethereum hourly net stats 3pm 4pm utc blocks27...
2     doggyswap eth btc cryptothis project one best ...
3     albt doesnt stop dex amm defi qnt vxv htr fwt ...
4     by buying tokens 100s property located chicago...
                            ...                        
95    membalas layer1crypto great project hope part ...
96    all apeswap buidl program f09fa4ab join publis...
97    btc eth doge future going pump next crypto soo...
98    while sideways action happening great time 2 a...
99    how get crypto heres got inf09fa4ad cryptosmar...
Name: Tweet, Length: 100, dtype: object

In [36]:
freq = pd.Series(' '.join(pandasTweet['Tweet']).split()).value_counts()[:10]
freq

eth         60
btc         30
crypto      19
bitcoin     17
doge        11
ethereum    11
nft         10
project     10
bnb          9
get          8
dtype: int64

In [37]:
freq = list(freq.index)
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
pandasTweet['Tweet'].head()

0    new coinmarketcap listing escointoken elg pric...
1    hourly net stats 3pm 4pm utc blocks270 avgdiff...
2    doggyswap cryptothis one best projects ive his...
3    albt doesnt stop dex amm defi qnt vxv htr fwt ...
4    by buying tokens 100s property located chicago...
Name: Tweet, dtype: object

In [38]:
freq = pd.Series(' '.join(pandasTweet['Tweet']).split()).value_counts()[-20:]
freq

mohfw_india            1
date                   1
46565                  1
155148                 1
2777089f09f9fa2        1
answer                 1
movers                 1
announcement           1
babydoge               1
dex                    1
cryptoanarchye280a6    1
followers              1
07                     1
cephalopod             1
alimila53316380        1
f09f9082               1
way                    1
offer                  1
tomhandy1              1
documents              1
dtype: int64

In [39]:
freq = list(freq.index)
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
pandasTweet['Tweet']

0     new coinmarketcap listing escointoken elg pric...
1     hourly net stats 3pm 4pm utc blocks270 avgdiff...
2     doggyswap cryptothis one best projects ive his...
3     albt doesnt stop amm defi qnt vxv htr fwt bsc ...
4     by buying tokens 100s property located chicago...
                            ...                        
95    membalas layer1crypto great hope part share ot...
96    all apeswap buidl program f09fa4ab join publis...
97    future going pump next soon telegram channel l...
98    while sideways action happening great time 2 a...
99    how heres got inf09fa4ad cryptosmart cryptocur...
Name: Tweet, Length: 100, dtype: object

In [40]:
pandasTweet

,Tweet date,Tweet,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,2021-06-07 16:01:28,new coinmarketcap listing escointoken elg pric...,10,155,14.600000,0,1,0,0
1,2021-06-07 16:01:21,hourly net stats 3pm 4pm utc blocks270 avgdiff...,12,145,11.166667,0,2,0,1
2,2021-06-07 16:01:11,doggyswap cryptothis one best projects ive his...,21,154,6.700000,7,3,0,2
3,2021-06-07 16:01:04,albt doesnt stop amm defi qnt vxv htr fwt bsc ...,19,157,7.315789,1,5,0,12
4,2021-06-07 16:01:04,by buying tokens 100s property located chicago...,20,158,6.950000,6,2,0,1
...,...,...,...,...,...,...,...,...,...
95,2021-06-07 15:40:25,membalas layer1crypto great hope part share ot...,21,156,6.476190,9,0,0,2
96,2021-06-07 15:39:47,all apeswap buidl program f09fa4ab join publis...,15,177,10.866667,2,2,0,1
97,2021-06-07 15:39:20,future going pump next soon telegram channel l...,23,125,4.681818,7,2,0,2
98,2021-06-07 15:38:54,while sideways action happening great time 2 a...,19,155,7.210526,5,0,1,1


In [41]:
pandasTweet['numerics'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
pandasTweet[['Tweet','numerics']].head()

,Tweet,numerics
0,new coinmarketcap listing escointoken elg pric...,2
1,hourly net stats 3pm 4pm utc blocks270 avgdiff...,0
2,doggyswap cryptothis one best projects ive his...,0
3,albt doesnt stop amm defi qnt vxv htr fwt bsc ...,0
4,by buying tokens 100s property located chicago...,0


In [45]:
pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('\d+', '', regex=True)
pandasTweet['Tweet']

0     new coinmarketcap listing escointoken elg pric...
1     hourly net stats pm pm utc blocks avgdiff ph h...
2     doggyswap cryptothis one best projects ive his...
3     albt doesnt stop amm defi qnt vxv htr fwt bsc ...
4     by buying tokens s property located chicago ff...
                            ...                        
95      membalas layercrypto great hope part share otea
96    all apeswap buidl program ffaab join publishx ...
97    future going pump next soon telegram channel l...
98    while sideways action happening great time  ac...
99    how heres got inffaad cryptosmart cryptocurren...
Name: Tweet, Length: 100, dtype: object

In [46]:
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: str(TextBlob(x).correct()))

In [47]:
from textblob import Word
nltk.download('wordnet')
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
pandasTweet['Tweet'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anubu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    new coinmarketcap listing escointoken leg pric...
1    hour net state pm pm etc block avgdiff oh hash...
2    doggyswap cryptothis one best project give his...
3    alba doesn stop am defy ant xv her fat bc we c...
4    by buying token s property located chicago uff...
Name: Tweet, dtype: object

In [63]:
TextBlob(pandasTweet['Tweet'][16]).ngrams(2)

[WordList(['price', 'change']),
 WordList(['change', 'h']),
 WordList(['h', 'market']),
 WordList(['market', 'cap']),
 WordList(['cap', 'banking'])]

In [65]:
tf1 = (pandasTweet['Tweet'][0:99]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,listing,1.0
1,leg,2.0
2,h,3.0
3,coinmarketcap,1.0
4,new,4.0
5,game,1.0
6,escointoken,1.0
7,price,11.0
8,volume,3.0
9,pm,2.0


In [69]:
import numpy as np
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(pandasTweet.shape[0]/(len(pandasTweet[pandasTweet['Tweet'].str.contains(word)])))
tf1

,words,tf,idf,tfidf
0,listing,1.0,4.605170,4.605170
1,leg,2.0,2.995732,5.991465
2,h,3.0,0.301105,0.903315
3,coinmarketcap,1.0,4.605170,4.605170
4,new,4.0,2.407946,9.631782
5,game,1.0,4.605170,4.605170
6,escointoken,1.0,4.605170,4.605170
7,price,11.0,2.207275,24.280024
8,volume,3.0,3.506558,10.519674
9,pm,2.0,4.605170,9.210340


In [70]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,listing,1.0,4.605170,4.605170
1,leg,2.0,2.995732,5.991465
2,h,3.0,0.301105,0.903315
3,coinmarketcap,1.0,4.605170,4.605170
4,new,4.0,2.407946,9.631782
5,game,1.0,4.605170,4.605170
6,escointoken,1.0,4.605170,4.605170
7,price,11.0,2.207275,24.280024
8,volume,3.0,3.506558,10.519674
9,pm,2.0,4.605170,9.210340


In [74]:
pandasTweet['sentiment'] = pandasTweet['Tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
pandasTweet[['Tweet','sentiment']]

,Tweet,sentiment
0,new coinmarketcap listing escointoken leg price h volume game,-0.131818
1,hour net state pm pm etc block avgdiff oh hashrate the avgsize b ten,0.000000
2,doggyswap cryptothis one best project give history cryptocurrency idea,1.000000
3,alba doesn stop am defy ant xv her fat bc we cake bake burgher un such inch,0.000000
4,by buying token s property located chicago uffa receive special designed bye,0.357143
5,new post top trader buying big dip altcoins cryptonews cryptocurrencyea,0.212121
6,limb clot halted biology technology future stock stock stockmarketea,0.000000
7,thanks elson must sent got back awesome dogecoin dogma,0.400000
8,part btcmarkets ch etc or coinspot a,0.000000
9,twicedaily thermometer mvrvz market value real value score sour spent output profit ratio,-0.016667


In [89]:
predictions = [1,0,1,1,1,1,1,1,1,0,1,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1]

In [90]:
#Evaluated the tweets and assigned 1 -> i it's positive 0 -> if it's negative/neutral
pandasTweet['Personal evalument'] = predictions

In [119]:
train = []
test = []
pred_train = []
pred_test = []

In [130]:
train = pandasTweet['Tweet'][0:80]
test = pandasTweet['Tweet'][80:100]
pred_train = pandasTweet['Personal evalument'][0:80]
pred_test = pandasTweet['Personal evalument'][80:100]

In [123]:
#Vectorizing
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pandasTweet['Tweet'])

xtrain_count =  count_vect.transform(train)
xtest_count =  count_vect.transform(test)
print (xtrain_count)

  (0, 110)	1
  (0, 193)	1
  (0, 236)	1
  (0, 255)	1
  (0, 320)	1
  (0, 332)	1
  (0, 383)	1
  (0, 443)	1
  (0, 600)	1
  (1, 38)	1
  (1, 39)	1
  (1, 41)	1
  (1, 59)	1
  (1, 194)	1
  (1, 260)	1
  (1, 278)	1
  (1, 379)	1
  (1, 402)	1
  (1, 434)	2
  (1, 518)	1
  (1, 540)	1
  (1, 546)	1
  (2, 52)	1
  (2, 137)	1
  (2, 145)	1
  :	:
  (77, 559)	1
  (77, 579)	1
  (77, 580)	1
  (78, 86)	1
  (78, 87)	1
  (78, 212)	1
  (78, 215)	1
  (78, 216)	1
  (78, 224)	1
  (78, 239)	1
  (78, 296)	1
  (78, 344)	1
  (78, 392)	1
  (78, 443)	1
  (78, 503)	1
  (78, 628)	1
  (79, 66)	1
  (79, 285)	1
  (79, 326)	1
  (79, 437)	1
  (79, 451)	1
  (79, 464)	1
  (79, 536)	1
  (79, 551)	1
  (79, 617)	1


In [127]:
#Training the model
clf = tree.DecisionTreeClassifier()
clf.fit(xtrain_count, pred_train)

DecisionTreeClassifier()

In [136]:
#Prediction
test_results = clf.predict(xtest_count)
test_results

array([1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
      dtype=int64)

In [131]:
#Predicted outcome:
pred_test

80    0
81    0
82    0
83    0
84    0
85    0
86    0
87    0
88    1
89    0
90    1
91    0
92    1
93    1
94    0
95    0
96    1
97    0
98    1
99    1
Name: Personal evalument, dtype: int64

In [133]:
testDataSet = pd.DataFrame(test)

In [137]:
testDataSet['Personal evaluation'] = pred_test
testDataSet['Test results'] = test_results

In [147]:
testDataSet

,Tweet,Personal evaluation,Test results
80,cooking something awesome iron finance zilliqa cell litecoin dash cry,0,1
81,obviously safemoon actually solving real problem average personedaefbf safemoonwallet sea,0,0
82,trying experiment different background love turned nut cleanest area,0,1
83,limit him could listed finance soon research add magic,0,1
84,a quiver investor option select favourite creature mint idea,0,1
85,unique modern within trading strategics developed directly trader company wanton,0,1
86,thanks mention kanhaiy gettibet_bsc good xefcdafabdbea,0,1
87,ff price update our hour uffa each uffa each dot uffa each a,0,0
88,ff price update u hour uffa uffa dot uffa a,1,0
89,dont forget time chain met donna blow dogecoin add magic such,0,1


In [141]:
print(classification_report(pred_test, test_results))

              precision    recall  f1-score   support

           0       0.43      0.23      0.30        13
           1       0.23      0.43      0.30         7

    accuracy                           0.30        20
   macro avg       0.33      0.33      0.30        20
weighted avg       0.36      0.30      0.30        20



In [170]:
#Percentage chance of increase in value of ETH
#According to my evaluation based on the tweets
predChance = 7/20 * 100
print(predChance)

35.0


In [ ]:
#Value should fall

In [171]:
#According to the model
predChanceTest = 13/20 * 100
print(predChanceTest)

65.0


In [172]:
#Value should increase
#Time of oldest Tweet within test group:
oldestDate

'2021-06-07 15:38:40'

In [173]:
#2810$
#Value at the time on most recent Tweet
recentDate

'2021-06-07 15:43:47'

In [ ]:
#2811